## seq2seq任务构建训练数据

1. 打开文件
2. 返回字符串

In [2]:
import os

data_dir='./../data/fra-eng'
with open(os.path.join(data_dir, 'fra.txt'), 'r',encoding='utf-8') as f:
    raw_text= f.read()
#字符串
print(type(raw_text))
print(raw_text[:75])

<class 'str'>
Go.	Va !
Hi.	Salut !
Run!	Cours !
Run!	Courez !
Who?	Qui ?
Wow!	Ça alors !



### 对文本的预处理
1. 特殊空格符号，统一空格
2. 统一小写
3. 单词和标点符号之间加上空格
4. 确定token级别是单词，不同token之间用空格分开

In [3]:
# text是文本字符串
def preprocess_nmt(text):
    """预处理“英语－法语”数据集"""

    # char是字符，prev_char是前一个字符
    def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '

    # 使用空格替换不间断空格
    # 使用小写字母替换大写字母
    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()

    # 在单词和标点符号之间插入空格
    #遍历字符串
    #如果当前字符是标点符号，并且上一个字符不是空格。那么在当前字符前面加空格
    #其他情况一律保留当前字符
    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char
           for i, char in enumerate(text)]
    return ''.join(out)

text = preprocess_nmt(raw_text)
print(type(text))
print(text[:80])

<class 'str'>
go .	va !
hi .	salut !
run !	cours !
run !	courez !
who ?	qui ?
wow !	ça alors !


### 文本字符串分割+token分割
- 输入text：文本字符串
- 输出：
  1. 源语言token为粒度的，序列
  2. 源语言token为粒度的，序列
- source：两层list, `[[token_1,...,token_n],...,[token_1,...,token_n]]`
- target：两层list, `[[token_1,...,token_n],...,[token_1,...,token_n]]`



In [4]:
# text：文本字符串
def tokenize_nmt(text, num_examples=None):
    #分割依据是：换行符、制表符、空格
    #换行符：不同序列
    #制表符：分割源语言和目标语言
    #空格：分割不同token
    source, target = [], []
    for i, line in enumerate(text.split('\n')):
        if num_examples and i > num_examples:
            break
        parts = line.split('\t')
        if len(parts) == 2:
            source.append(parts[0].split(' '))
            target.append(parts[1].split(' '))
    return source, target

#source, target都是双层list，里面一层的list是不定长度的序列
#token是单词，包括标点符号
source_tokens, target_tokens = tokenize_nmt(text)
source_tokens[:6], target_tokens[:6]

([['go', '.'],
  ['hi', '.'],
  ['run', '!'],
  ['run', '!'],
  ['who', '?'],
  ['wow', '!']],
 [['va', '!'],
  ['salut', '!'],
  ['cours', '!'],
  ['courez', '!'],
  ['qui', '?'],
  ['ça', 'alors', '!']])

In [19]:
import torch
from torch.utils.data import Dataset,DataLoader
from vocabulary import Vocab

class SequenceDataset(Dataset):
    def __init__(self,tokens,num_steps=10,min_freq=2,reserved_tokens=['<pad>', '<bos>', '<eos>']):
        self.tokens = tokens
        self.vocab = Vocab(self.tokens, min_freq=min_freq,reserved_tokens =reserved_tokens)
        self.truncate_pad_tensor, self.valid_len, = self.truncate_pad(num_steps)
    
    def __len__(self):
        return self.truncate_pad_tensor.shape[0]
    
    def __getitem__(self,index):
        return (self.truncate_pad_tensor[index], self.valid_len[index])
    
    
    def truncate_pad(self,num_steps,ending_token='<eos>', padding_token='<pad>'):
        sequences_idx = [self.vocab[sequence] for sequence in self.tokens]
        #每个序列后面加一个'<eos>'索引
        sequences_idx = [idx + [self.vocab[ending_token]] for idx in sequences_idx]
        # 每个序列截断填充处理
        truncate_pad=(
            [sequence_idx[:num_steps] if len(sequence_idx)>num_steps 
            else sequence_idx + [self.vocab[padding_token]] * (num_steps - len(sequence_idx)) 
            for sequence_idx in sequences_idx])

        # 形状：(s,num_steps)
        # 其中s是序列数目，num_steps是固定的序列长度
        truncate_pad_tensor = torch.tensor(truncate_pad)
        # 有效长度
        valid_len = (truncate_pad_tensor != self.vocab['<pad>']).type(torch.int32).sum(1)
        return (truncate_pad_tensor, valid_len)

class Seq2seqDataset(Dataset):
    def __init__(self,source_dataset,target_dataset):
        self.source_dataset = source_dataset
        self.target_dataset = target_dataset

    def __len__(self):
        assert len(self.source_dataset)==len(self.target_dataset)
        return (len(self.source_dataset))
    
    def __getitem__(self,index):
        return (self.source_dataset[index][0],self.source_dataset[index][1],
        self.target_dataset[index][0],self.target_dataset[index][1])

source_dataset = SequenceDataset(source_tokens)
target_dataset = SequenceDataset(target_tokens)

seq2seq_dataset = Seq2seqDataset(source_dataset,target_dataset)
seq2seq_dataset[0]

data_iter= DataLoader(seq2seq_dataset, batch_size=2, shuffle=True)
for X, X_valid_len, Y, Y_valid_len in data_iter:
    print('X:', X.type(torch.int32))
    print('X的有效长度:', X_valid_len)
    print('Y:', Y.type(torch.int32))
    print('Y的有效长度:', Y_valid_len)
    break


X: tensor([[159,  13, 108,   6, 828,   9,   3,   1,   1,   1],
        [  5,  36,  45,  35, 357,   7,  25, 294,  23, 113]], dtype=torch.int32)
X的有效长度: tensor([ 7, 10])
Y: tensor([[  58,   41, 4326,  922,    7,    3,    1,    1,    1,    1],
        [   5,   78,    9,   22,   11,  344,    8,   10,   65,  293]],
       dtype=torch.int32)
Y的有效长度: tensor([ 6, 10])
